In [ ]:
import json
import sqlite3
import os
import dotenv
import  yt_dlp
from datetime import datetime, timezone    
import calendar 
from joblib import Parallel, delayed

In [2]:
dotenv.load_dotenv()

True

In [ ]:
def dbconnection():
    # Connect to a local SQLite database file
    conn = sqlite3.connect('fitlabeler.db')
    return conn

In [4]:
def write_json(data, filename='data.json'): 
    with open('Data/'+filename  ,'w') as f: 
        json.dump(data, f)


In [19]:
def getchannels(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM YT_CHANNELS where status =0")
    channels = cursor.fetchall()
    cursor.close()
    return channels

In [20]:
def getallchannels(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM YT_CHANNELS where status =0")
    channels = cursor.fetchall()
    cursor.close()
    return channels

In [8]:
def insertchanneldetails(connection,channel):
    cursor = connection.cursor()
    write_json(channel,channel['id']+'.json')
    with open('Data/'+channel['id']+'.json', 'rb') as f:
        cursor.execute("INSERT INTO YT_CHANNEL_DETAILS (CHANNELID, AVAILABILITY, CHANNEL_FOLLOWER_COUNT,PAYLOAD) VALUES (:1, :2, :3, :4)", (channel['id'], channel['availability'], channel['channel_follower_count'], f.read()))
    cursor.close()


In [ ]:
def updatechannelstatus(connection,channel):
    cursor = connection.cursor()
    cursor.execute("UPDATE YT_CHANNELS SET STATUS = 1 WHERE CHANNELID = ?", (channel[0],))
    cursor.close()

In [ ]:
def insertvideodetails(connection,video,channelid):
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM YT_VIDEOS WHERE VIDEOID = ?", (video['id'],))
    row=cursor.fetchone()
    if row is None:
            cursor.execute("INSERT INTO YT_VIDEOS (VIDEOID, CHANNELID, VIEW_COUNT,DURATION,PAYLOAD) VALUES (?, ?, ?, ?, ?)", (video['id'], channelid,video['view_count'],video['duration'], json.dumps(video)))
    else:
        cursor.execute("UPDATE YT_VIDEOS SET VIEW_COUNT = :1, DURATION = :2, PAYLOAD = :3, UPDATEDATE=SYSDATE WHERE VIDEOID = ?", (video['view_count'],video['duration'],json.dumps(video),video['id']))
    cursor.close()

In [36]:
def processchannels(channels):
    connection = dbconnection()
    i=0
    for channel in channels:
        ydl_opts = {
                'extract_flat': True,
                'quiet': True,
            }
        i=i+1
        channel_url= f"https://www.youtube.com/channel/{channel[0]}/videos"
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            try:
                info = ydl.extract_info(channel_url, download=False)
                #print(json.dumps(info, indent=4))  
                insertchanneldetails(connection,info)
                #list_of_videos_urls = [entry['id'] for entry in info['entries']]
                for entry in info['entries']:
                    insertvideodetails(connection,entry,channel[0])
            except Exception as e:
                print(e)
                continue
        updatechannelstatus(connection,channel)
        if(i%10==0):
            connection.commit()
    connection.commit()
    connection.close()

In [39]:
connection = dbconnection()
rows=getallchannels(connection)
chunk_size = 10
chunks = [rows[i: i+chunk_size] for i in range(0, len(rows), chunk_size)]
Parallel(n_jobs=10)(delayed(processchannels)(chunks[row]) for row in range(0, len(chunks)))
connection.close()

ERROR: [youtube:tab] UC0cicvLSgneLaK-sSlkiwWA: This channel does not have a videos tab
ERROR: [youtube:tab] UCVR8zDuJIm44DL9l2Zr0wwQ: This channel does not have a videos tab


ERROR: [youtube:tab] UC0cicvLSgneLaK-sSlkiwWA: This channel does not have a videos tab
ERROR: [youtube:tab] UCVR8zDuJIm44DL9l2Zr0wwQ: This channel does not have a videos tab


ERROR: [youtube:tab] UC9JdlOrPLM5FQ26pc2PS5Ag: This channel does not have a videos tab


ERROR: [youtube:tab] UC9JdlOrPLM5FQ26pc2PS5Ag: This channel does not have a videos tab


ERROR: [youtube:tab] UC6GwiWH0hUkkva_ACzINgBQ: This channel does not have a videos tab


...

ERROR: [youtube:tab] UC_zRjo3rhHd6EJd3018FoiQ: This channel does not have a videos tab


ERROR: [youtube:tab] UC_zRjo3rhHd6EJd3018FoiQ: This channel does not have a videos tab


ERROR: [youtube:tab] UCfY07_SkRFnv3-eFyzTeEAg: This channel does not have a videos tab


ERROR: [youtube:tab] UCfY07_SkRFnv3-eFyzTeEAg: This channel does not have a videos tab


KeyboardInterrupt: 